In [ ]:
# importacion general de librerias y de visualizacion (matplotlib y seaborn)
import pandas as pd
import numpy as np
import random
import re

pd.options.display.float_format = '{:20,.2f}'.format # suprimimos la notacion cientifica en los outputs

import warnings
warnings.filterwarnings('ignore')

In [ ]:
train_data = pd.read_csv('~/Documents/Datos/DataSets/TP2/train_featured.csv')
test_data = pd.read_csv('~/Documents/Datos/DataSets/TP2/test_featured.csv')
test_data['clean_text'].fillna("", inplace=True)

In [ ]:
location_dic = {}
for location in train_data.location_original:
    if (location != location): continue
    try:
        location_dic[location] += 1
    except KeyError:
        location_dic[location] = 1
    except TypeError:
        pass
location_dic

In [ ]:
location_dic = {k: v for k, v in sorted(location_dic.items(), key=lambda item: item[1], reverse=True)}
location_dic = {k: v for k, v in location_dic.items() if v >= 10}
location_dic

In [ ]:
for x in location_dic.keys():
    train_data[x] = 0
    test_data[x] = 0

In [ ]:
def one_hot_locations(row):
    loc = row['location_original']
    try:
        if loc in row:
            row[loc] = 1
    except AttributeError:
        pass
    return row

In [ ]:
train_data = train_data.apply(one_hot_locations, axis=1)
train_data.head()

In [ ]:
test_data = test_data.apply(one_hot_locations, axis=1)
test_data.head()

In [ ]:
hashtag_dic = {}
for ls in train_data.hashtags.str.split():
    try:
        for hashtag in ls:
            try:
                hashtag_dic[hashtag] += 1
            except KeyError:
                hashtag_dic[hashtag] = 1
    except TypeError:
        pass

In [ ]:
hashtag_dic = {k: v for k, v in sorted(hashtag_dic.items(), key=lambda item: item[1], reverse=True)}
hashtag_dic = {k: v for k, v in hashtag_dic.items() if v >= 10}
hashtag_dic

In [ ]:
for x in hashtag_dic.keys():
    train_data[x] = 0
    test_data[x] = 0

In [ ]:
def one_hot_hashtags(row):
    try:
        for x in row.hashtags.split():
            if x in row:
                row[x] = 1
    except AttributeError:
        pass
    return row

In [ ]:
train_data = train_data.apply(one_hot_hashtags, axis=1)
train_data.head()

In [ ]:
test_data = test_data.apply(one_hot_hashtags, axis=1)
test_data.head()

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text  import TfidfVectorizer
from sklearn.metrics import accuracy_score
import xgboost as xgb
from sklearn.preprocessing import OneHotEncoder

In [ ]:
train_data.drop(["id_original","keyword_original","location_original","text_original","hashtags","labels","clean_text"], axis=1, inplace=True)
real_test_data = test_data.drop(["id_original","keyword_original","location_original","text_original","hashtags","labels","clean_text"], axis=1)

In [ ]:
train, test = train_test_split(train_data, test_size=0.2)

In [ ]:
model_xgb = xgb.XGBClassifier(n_estimators=300, colsample_bytree=0.5, learning_rate=0.1, max_depth=11)
model_xgb.fit(train.drop(['target_label'], axis=1), train.target_label)

In [ ]:
test_prediction = model_xgb.predict(test.drop(['target_label'], axis=1))
print("Accuracy score: %f" % (accuracy_score(test.target_label, test_prediction)))

In [ ]:
pd.DataFrame(model_xgb.feature_importances_, index=train.drop(['target_label'], axis=1).columns, columns=["importancia"]).\
        sort_values(by="importancia",ascending=False)

In [ ]:
model_xgb = xgb.XGBClassifier(n_estimators=300, colsample_bytree=0.5, learning_rate=0.1, max_depth=11)
model_xgb.fit(train_test.drop(['target_label'], axis=1), train_test.target_label)

In [ ]:
real_test_prediction = model_xgb.predict(real_test_data)
test_data['target'] = real_test_prediction
test_data[['id_original', 'target']].rename(columns={'id_original': 'id'}).to_csv('~/Documents/Datos/DataSets/TP2/res_XGB_1.csv', index=False)